In [1]:
import pandas as pd
import glob, os
pd.set_option('float_format', '{:f}'.format)

dateparse = lambda x: pd.datetime.strptime(x, '%Y%m%d')

pd.options.display.max_columns = 999
pd.options.display.max_rows = 100

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [2]:
def plot_chart(df):
    # рисуем свечной график с объемами

    #INCREASING_COLOR = '#17BECF'
    #DECREASING_COLOR = '#7F7F7F'
    
    ticker_name = df.Ticker[0]
    data = [ dict(
        type = 'candlestick',
        open = df.Open,
        high = df.High,
        low = df.Low,
        close = df.Close,
        x = df.Date,
        yaxis = 'y2',
        name = ticker_name,
    #    increasing = dict( line = dict( color = INCREASING_COLOR ) ),
    #    decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
    ) ]

    layout=dict()

    fig = dict(data=data, layout=layout)

    fig['layout'] = dict()
    #fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
    fig['layout']['xaxis'] = dict(rangeselector = dict(visible = True), rangeslider = dict(visible = False))
    fig['layout']['yaxis'] = dict(domain = [0, 0.2], showticklabels = False, autorange=True)
    fig['layout']['yaxis2'] = dict(domain = [0.2, 0.8], autorange=True)
    fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
    fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )

    fig['data'].append( dict( x=df.Date, y=df.Volume,  
    #                         marker=dict( color=colors ),
                             type='bar', yaxis='y', name='Volume') )
    plotly.offline.iplot(fig)
    return


In [3]:
path = r'C:\Users\parsh\Desktop\Exchange\data\US_stocks_csv'
file_name = '/NYSE/TDW.csv'
file_ticker = path + file_name

In [4]:
vol_df = pd.read_csv(file_ticker)

In [5]:
vol_df['Date'] = pd.to_datetime(vol_df['Date'], format='%Y%m%d', errors='ignore')

In [6]:
vol_df.head()

,Ticker,Date,Open,High,Low,Close,Volume
0,TDW,1985-01-02,620.432430,628.286010,608.652040,612.578860,751
1,TDW,1985-01-03,612.578860,612.578860,596.871700,596.871700,3040
2,TDW,1985-01-04,600.798460,600.798460,581.164550,581.164550,2973
3,TDW,1985-01-07,596.871700,608.652040,596.871700,608.652040,2037
4,TDW,1985-01-08,600.798460,612.578860,589.018130,596.871700,1550


# Turtle simple soup

In [7]:
def plot_signal(df, signal_name, stop_legend = False, opacity_cndl = 0.8):
    # рисуем свечной график с объемами
    # signal_name - колонка с сигналом
    # ниже кода для трех состояний сигнала: +1, -1, 0
    # +1 синий, -1 розовый, 0 цвет свечек не меняется

    df['Date_2'] = df['Date'].dt.strftime('%d-%m-%Y')

    color_signal_long = '#0028FF' # цвет сигнала для лонга - синий
    color_signal_short = '#ff00f1' # цвет сигнала для шорта - розовый
    
    mask_long = df[signal_name]==True
#    mask_short = df[signal_name]<0
        
    name_long = 'Long_' + str(round(df[mask_long].shape[0]*100/df.shape[0],1)) + '%'
#    name_short = 'Short_' + str(round(df[mask_short].shape[0]*100/df.shape[0],1)) + '%'
        
    ticker_name = df.Ticker[0]
    data = [ dict(
        name = ticker_name + '_' + signal_name,
        type = 'candlestick',
        x = df['Date_2'],
        yaxis = 'y2',
        open = df['Open'],
        high = df['High'],
        low = df['Low'],
        close = df['Close'],
        opacity = opacity_cndl
    ),
        dict(
        name = name_long,
        type = 'candlestick',
        x = df['Date_2'][mask_long],
        yaxis = 'y2',
        open = df['Open'][mask_long],
        high = df['High'][mask_long],
        low = df['Low'][mask_long],
        close = df['Close'][mask_long],
        hoverinfo='none',
        increasing = dict(line = dict(color = '#0028FF')),
        decreasing = dict(line = dict(color = '#0028FF'))
    )
           ]

    layout=dict()

    fig = dict(data=data, layout=layout)

    fig['layout'] = dict()
    #fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
    #categoryorder = 'category ascending', 
    fig['layout']['xaxis'] = dict(type ='category', 
                                  tickfont=dict(size=10),
                                  rangeselector = dict(visible = True), 
                                  rangeslider = dict(visible = False))
    fig['layout']['yaxis'] = dict(domain = [0, 0.2], 
                                  showticklabels = False, 
                                  autorange = True, 
                                  type = 'log',
#                                  title = 'Volume'
                                 )
    fig['layout']['yaxis2'] = dict(domain = [0.1, 0.95], 
                                   autorange = True, 
                                   type = 'log',
                                   title = ticker_name + ' prices')
    fig['layout']['legend'] = dict(orientation = 'h', 
                                   y = 0.95, x = 0.1, 
                                   yanchor = 'bottom')
    fig['layout']['margin'] = dict(t = 40, b = 40, r = 40, l = 40)
    
    if stop_legend:
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sl_long'], type='scatter', yaxis='y2', 
                                line = dict(width = 1), marker=dict(color='#FF4500'), name='SL_long'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sl_short'], type='scatter', yaxis='y2', 
                            line = dict(width = 1), marker=dict(color='#FF4500'), name='SL_short'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sp_long'], type='scatter', yaxis='y2', 
                                line = dict(width = 1), marker=dict(color='#008B8B'), name='SP_long'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sp_short'], type='scatter', yaxis='y2', 
                            line = dict(width = 1), marker=dict(color='#008B8B'), name='SP_short'))


    plotly.offline.iplot(fig)
    return


In [8]:
vol_df['max_16_days'] = vol_df['High'].shift(4).rolling(16).max()
vol_df['max_3_days'] = vol_df['High'].shift(1).rolling(3).max()

In [9]:
vol_df['false_break_1_condition_1'] = vol_df['max_16_days'] > vol_df['max_3_days']

In [10]:
vol_df['false_break_1_condition_2'] = vol_df['High'] > vol_df['max_16_days']

In [11]:
vol_df['false_break_1'] = vol_df['false_break_1_condition_1'] & vol_df['false_break_1_condition_2']

In [12]:
vol_df[vol_df['false_break_1'] == True].head()

,Ticker,Date,Open,High,Low,Close,Volume,max_16_days,max_3_days,false_break_1_condition_1,false_break_1_condition_2,false_break_1
31,TDW,1985-02-14,604.725280,620.432430,589.018130,596.871700,3709,604.725280,596.871700,True,True,True
61,TDW,1985-03-29,541.896670,581.164550,541.896670,581.164550,1853,577.237790,549.750240,True,True,True
138,TDW,1985-07-19,471.214510,479.068080,459.434140,471.214510,487,467.287720,459.434140,True,True,True
143,TDW,1985-07-26,490.848450,518.335940,463.360930,502.628810,1744,479.068080,471.214510,True,True,True
165,TDW,1985-08-27,463.360930,486.921660,463.360930,486.921660,3206,475.141300,471.214510,True,True,True


In [13]:
mask = (vol_df['Date'] >= '2013-09-16') & (vol_df['Date'] <= '2013-09-19')
print(vol_df.loc[mask])

     Ticker       Date        Open        High         Low       Close  \
7237    TDW 2013-09-16 1816.060670 1827.998170 1806.636350 1824.542600   
7238    TDW 2013-09-17 1824.856690 1847.789180 1824.856690 1842.134520   
7239    TDW 2013-09-18 1844.019410 1864.438720 1821.715210 1860.983150   
7240    TDW 2013-09-19 1862.868040 1872.920530 1854.072020 1864.438720   

      Volume  max_16_days  max_3_days  false_break_1_condition_1  \
7237    5994  1826.427370 1831.767820                      False   
7238    5622  1831.767820 1827.998170                       True   
7239    6787  1831.767820 1847.789180                      False   
7240    5402  1831.767820 1864.438720                      False   

      false_break_1_condition_2  false_break_1  
7237                       True          False  
7238                       True           True  
7239                       True          False  
7240                       True          False  


In [15]:
plot_signal(vol_df, signal_name = 'false_break_1')

# Turtle soup +1

In [ ]:
vol_df['max_17_days'] = vol_df['High'].shift(3).rolling(17).max()
vol_df['max_2_days'] = vol_df['High'].shift(2).rolling(2).max()

In [ ]:
vol_df['ex_candle'] = vol_df['High'].shift(1)

In [ ]:
vol_df['false_break_2_condition_1'] = vol_df['max_17_days'] > vol_df['max_2_days']
vol_df['false_break_2_condition_2'] = vol_df['ex_candle'] > vol_df['max_17_days']
vol_df['false_break_3_condition_3'] = vol_df['High'] <= vol_df['max_17_days']

In [ ]:
vol_df['false_break_2'] = vol_df['condition_1'] & vol_df['condition_2'] & vol_df['condition_3']

In [ ]:
vol_df[vol_df['false_break_2']  == True].head()

In [ ]:
mask = (vol_df['Date'] >= '1987-08-03') & (vol_df['Date'] <= '1987-08-08')
print(vol_df.loc[mask][['Date', 'High', 'max_17_days', 'max_2_days', 'ex_candle',  'false_break_2_condition_1', 'false_break_2_condition_2', 'false_break_3_condition_3', 'false_break_2']])

In [ ]:
plot_chart(vol_df)